# from formulas import *

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf

In [ ]:
def plot_series(time, series, format="-", start=0, end=None, label=None):
    plt.plot(time[start:end], series[start:end], format, label=label)
    plt.xlabel("Time")
    plt.ylabel("Value")
    if label:
        plt.legend(fontsize=14)
    plt.grid(True)


keras = tf.keras

df = pd.read_csv('maayan/stocks_data_30_6.csv')
df['Date'] = pd.to_datetime(df['Date'])


df.tail(100)

df.dropna(inplace=True)

series = df['Close']

# Create train data set

In [ ]:
test_split_date = '2005-09-16'
test_split_index = np.where(df.Date == test_split_date)[0][0]
x_test = df.loc[df['Date'] >= test_split_date]['Close']

# Create valid data set

In [ ]:
valid_split_index = (train_split_index.max(),test_split_index.min())
x_valid = df.loc[(df['Date'] < test_split_date) & (df['Date'] > train_split_date)]['Close']

# set style of charts

In [ ]:
sns.set(style="darkgrid")
plt.rcParams['figure.figsize'] = [10, 10]

# Create a plot showing the split of the train, valid, and test data.

In [ ]:
plt.plot(x_train, label = 'Train')
plt.plot(x_valid, label = 'Validate')
plt.plot(x_test, label = 'Test')
plt.title('Train Valid Test Split of Data')
plt.ylabel('Dollars $')
plt.xlabel('Timestep in Days')
plt.legend()
print(x_train.index.max(),x_valid.index.min(),x_valid.index.max(),x_test.index.min(),x_test.index.max())

plt.show()

model_mae_scores = {}

# Plot chart with all details untouched

In [ ]:
plot_series(time=df.index,series=df['Close'], label = 'TSLA Close Price')
plt.ylabel('Dollars $')
plt.xlabel('Timestep in Days')
plt.title('Price History of TSLA Jan-1993 to Dec-2020')


naive_forecast = series[test_split_index-1 :-1]

plt.figure(figsize=(10, 6))
plot_series(x_test.index, x_test, label="Actual")
plot_series(x_test.index, naive_forecast, label="Forecast")
plt.ylabel('Dollars $')
plt.xlabel('Timestep in Days')
plt.title('Naive Forecast vs Actual')
plt.show()

# Alternative way to show MAE to stay consistent with what we will be doing later

In [ ]:
naive_forecast_mae = keras.metrics.mean_absolute_error(x_test, naive_forecast).numpy()
model_mae_scores['naive'] = naive_forecast_mae

# view the dictionary of mae scores

In [ ]:
print(model_mae_scores)

# Show first 3 values of our forecast

In [ ]:
print(x_test.values[:3])

# Calculate and show first 3 values of the calculated error (MAE)

In [ ]:
print('*'*100)
print(np.abs(naive_forecast[:3].values - x_test[:3].values))

# Choose a window size for the moving average

In [ ]:
window = 20

# Create a moving average over the entire dataset

In [ ]:
moving_avg = df['Close'].rolling(window=window).mean()

# Slice the moving average on the forecast

In [ ]:
moving_avg_forecast = moving_avg.values[test_split_index - window:df.index.max() - window + 1]

plt.figure(figsize=(10, 6))
plot_series(x_test.index, x_test, label="Series")
plot_series(x_test.index, moving_avg_forecast, label="Moving average (20 days)")
plt.ylabel('Dollars $')
plt.xlabel('Timestep in Days')
plt.title('SMA20 vs Actual')
plt.show()

ma_20 = keras.metrics.mean_absolute_error(x_test, moving_avg_forecast).numpy()

model_mae_scores['SMA 20'] = ma_20
print(model_mae_scores)